### 1. 데이터 준비 및 전처리
- **압축 해제**: `mini_data.zip` 파일의 내용을 압축 해제하고 파일 구조를 확인합니다.
- **데이터 분리**: 이미지와 마스크를 연결하는 CSV 파일을 사용하여 이미지와 해당 마스크를 매칭하고, 이를 훈련 데이터와 검증 데이터로 9:1 비율로 분리합니다.

In [1]:
import os
import pandas as pd

data_path = '../DataSet/BeadSegmentation/'
# train.csv 파일 경로
csv_path = os.path.join(data_path, 'mini_data', 'train.csv')

# CSV 파일 로드
data_df = pd.read_csv(csv_path)
data_df.head()


FileNotFoundError: [Errno 2] No such file or directory: '../DataSet/BeadSegmentation/mini_data/train.csv'

### 데이터 분리 단계
- **데이터셋 셔플링**: 데이터를 무작위로 섞어서 모델이 특정 순서에 의존하지 않도록 합니다.
- **분리**: 전체 데이터셋을 90%는 훈련용으로, 나머지 10%는 검증용으로 분리합니다.
- **파일 저장**: 분리된 데이터를 각각의 폴더(훈련 및 검증)에 저장합니다.

In [58]:
from sklearn.model_selection import train_test_split

# 데이터셋을 훈련용과 검증용으로 분리 (90% 훈련, 10% 검증)
train_df, val_df = train_test_split(data_df, test_size=0.1, random_state=42)

# 훈련용과 검증용 데이터 프레임 확인
train_df.head(), val_df.head(), len(train_df), len(val_df)


(                         ImageId                             MaskId
 37  20230930_21h03m42s_B11_2.png  20230930_21h03m42s_B11_2_mask.png
 12  20230930_21h03m13s_B04_3.png  20230930_21h03m13s_B04_3_mask.png
 19  20230930_21h03m20s_B06_2.png  20230930_21h03m20s_B06_2_mask.png
 4   20230930_21h03m06s_B02_3.png  20230930_21h03m06s_B02_3_mask.png
 25  20230930_21h03m31s_B08_2.png  20230930_21h03m31s_B08_2_mask.png,
                          ImageId                             MaskId
 27  20230930_21h03m31s_B08_4.png  20230930_21h03m31s_B08_4_mask.png
 40  20230930_21h03m46s_B12_1.png  20230930_21h03m46s_B12_1_mask.png
 26  20230930_21h03m31s_B08_3.png  20230930_21h03m31s_B08_3_mask.png
 43  20230930_21h03m46s_B12_4.png  20230930_21h03m46s_B12_4_mask.png
 24  20230930_21h03m31s_B08_1.png  20230930_21h03m31s_B08_1_mask.png,
 43,
 5)

### 데이터셋 폴더 셋업

In [59]:
import os
import shutil

# 데이터 세트 유형 선택 ('mini_data' 또는 'original_data')
data_type = 'mini_data'  # 또는 'original_data'

# 데이터셋 경로 설정
dataset_path = os.path.join(data_path, data_type)

# 훈련 및 검증 데이터 폴더 생성
train_dir = os.path.join(data_path, 'train')
val_dir = os.path.join(data_path, 'val')

# 훈련 및 검증 폴더 내의 이미지와 마스크 폴더 생성
train_img_dir = os.path.join(train_dir, 'images')
train_mask_dir = os.path.join(train_dir, 'masks')
val_img_dir = os.path.join(val_dir, 'images')
val_mask_dir = os.path.join(val_dir, 'masks')

# 기존 폴더가 있을 경우 삭제
for dir_path in [train_dir, val_dir]:
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
    os.makedirs(dir_path)

# 이미지와 마스크 폴더 생성
for sub_dir in [train_img_dir, train_mask_dir, val_img_dir, val_mask_dir]:
    os.makedirs(sub_dir)

# 훈련용 데이터 복사
for idx, row in train_df.iterrows():
    # 원본 이미지와 마스크 경로
    src_image_path = os.path.join(dataset_path, 'images', row['ImageId'])
    src_mask_path = os.path.join(dataset_path, 'masks', row['MaskId'])
    # 대상 이미지와 마스크 경로
    dst_image_path = os.path.join(train_img_dir, row['ImageId'])
    dst_mask_path = os.path.join(train_mask_dir, row['MaskId'])
    
    # 파일 복사
    shutil.copy(src_image_path, dst_image_path)
    shutil.copy(src_mask_path, dst_mask_path)

# 검증용 데이터 복사
for idx, row in val_df.iterrows():
    # 원본 이미지와 마스크 경로
    src_image_path = os.path.join(dataset_path, 'images', row['ImageId'])
    src_mask_path = os.path.join(dataset_path, 'masks', row['MaskId'])
    # 대상 이미지와 마스크 경로
    dst_image_path = os.path.join(val_img_dir, row['ImageId'])
    dst_mask_path = os.path.join(val_mask_dir, row['MaskId'])
    
    # 파일 복사
    shutil.copy(src_image_path, dst_image_path)
    shutil.copy(src_mask_path, dst_mask_path)

# 확인
os.listdir(train_img_dir), os.listdir(train_mask_dir), os.listdir(val_img_dir), os.listdir(val_mask_dir)


(['20230930_21h03m13s_B04_2.png',
  '20230930_21h03m13s_B04_4.png',
  '20230930_21h03m20s_B06_3.png',
  '20230930_21h03m50s_B13_2.png',
  '20230930_21h03m35s_B09_1.png',
  '20230930_21h03m35s_B09_3.png',
  '20230930_21h03m06s_B02_3.png',
  '20230930_21h03m39s_B10_3.png',
  '20230930_21h03m09s_B03_3.png',
  '20230930_21h03m50s_B13_3.png',
  '20230930_21h03m46s_B12_3.png',
  '20230930_21h03m06s_B02_4.png',
  '20230930_21h03m42s_B11_1.png',
  '20230930_21h03m20s_B06_4.png',
  '20230930_21h03m39s_B10_2.png',
  '20230930_21h03m06s_B02_1.png',
  '20230930_21h03m20s_B06_1.png',
  '20230930_21h03m50s_B13_1.png',
  '20230930_21h03m42s_B11_3.png',
  '20230930_21h03m09s_B03_2.png',
  '20230930_21h03m06s_B02_2.png',
  '20230930_21h03m42s_B11_4.png',
  '20230930_21h03m35s_B09_2.png',
  '20230930_21h03m03s_B01_2.png',
  '20230930_21h03m16s_B05_4.png',
  '20230930_21h03m13s_B04_1.png',
  '20230930_21h03m35s_B09_4.png',
  '20230930_21h03m42s_B11_2.png',
  '20230930_21h03m16s_B05_3.png',
  '20230930_21